# NewYork Taxi Duration Prediction

## Preprocessing Data ##

In [1]:
# import libraries for data preprocessing

import pandas as pd
from sklearn.feature_extraction import DictVectorizer

import os

In [11]:
import mlflow

mlflow_version = mlflow.__version__
print("MLflow version:", mlflow_version)

MLflow version: 2.3.2


In [2]:
# function to read the data and remove outliers

def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [3]:
# function to preprocess the dataset

def preprocess(df: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

In [4]:
# call read and preprocessing functions for train, validation and test datasets

def run(raw_data_path: str, dest_path: str, dataset: str = "green"):
    # load parquet files
    df_train = read_dataframe(
        os.path.join(raw_data_path, f"{dataset}_tripdata_2022-01.parquet")
    )
    df_valid = read_dataframe(
        os.path.join(raw_data_path, f"{dataset}_tripdata_2022-02.parquet")
    )
    df_test = read_dataframe(
        os.path.join(raw_data_path, f"{dataset}_tripdata_2022-03.parquet")
    )

    # extract the target
    target = 'duration'
    y_train = df_train[target].values
    y_valid = df_valid[target].values
    y_test = df_test[target].values

    # fit the dictvectorizer and preprocess data
    dv = DictVectorizer()
    X_train, dv = preprocess(df_train, dv, fit_dv=True)
    X_valid, _ = preprocess(df_valid, dv, fit_dv=False)
    X_test, _ = preprocess(df_test, dv, fit_dv=False)

    return X_train, y_train, X_valid, y_valid, X_test, y_test, dv

In [5]:
# set the paths
raw_data_path = r'/home/ubuntu/notebooks/data_2022'
# raw_data_path = r'C:\Users\bhaga\Documents\MLOps\Zoomcamp\MLOps_Zoomcamp\Week2\data'
dest_path = r'/home/ubuntu/notebooks/data_2022/models' # r'./output'
dataset = r'green'

In [6]:
print('hello world')

hello world


In [7]:
!pwd

/home/ubuntu/notebooks


In [13]:
# get the preprocessed datasets

X_train, y_train, X_valid, y_valid, X_test, y_test, dv = \
    run(raw_data_path=raw_data_path, dest_path=dest_path, dataset=dataset)

In [14]:
dv

DictVectorizer()

In [15]:
import pickle


In [16]:
with open('dictvectorizer.pkl', 'wb') as file:
    pickle.dump(dv, file)

In [17]:
import os

file_size = os.path.getsize('dictvectorizer.pkl')
print("Size of the DictVectorizer file:", file_size, "bytes")

Size of the DictVectorizer file: 153660 bytes


### Import and set mlflow ###

In [19]:
import mlflow                                     # include mlflow to track

mlflow.set_tracking_uri("sqlite:///mlflow.db")    # since sqlite is used as mlflow backend
mlflow.set_experiment("nyc-taxi-experiment")      # set the experiment name. Created if not existing 

<Experiment: artifact_location='/home/ubuntu/notebooks/mlruns/1', creation_time=1685901657554, experiment_id='1', last_update_time=1685901657554, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

<img src="img/01trackerpage.png" />

### Train using Linear Regressor, Add experiment to mlflow manually ###

In [20]:
# import linear regression model to train

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [21]:
# create models folder if not existing already

dest_path = 'models'
os.makedirs(dest_path, exist_ok=True)

In [22]:
# folder_path = r'./output'
# file_names = os.listdir(folder_path)

# for file_name in file_names:
#     print(file_name)

In [42]:
# train the model and track the run in mlflow

with mlflow.start_run():

    mlflow.set_tag("developers", "cristian & bhagabat") # set the tags to the run as key value pair

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv") # add param log
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv") # add param log

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_valid)
    rmse = mean_squared_error(y_valid, y_pred, squared=False)

    mlflow.log_metric("rmse", rmse) # log metric
    mlflow.log_artifact(local_path= dest_path, artifact_path="models_pickle") # log model artifact
#     r'/home/ubuntu/notebooks/models/lin_reg.bin'

In [43]:
import os
os.getcwd()

'/home/ubuntu/notebooks'

<img src="img/02lrexperiment.png" />

### Train using XGBOOST and Manually track experiment with hyperparameter tunning ###

In [44]:
import xgboost as xgb

In [45]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [46]:
# change dataset format to DMatrix as per XGBOOST required format

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_valid, label=y_valid)

In [47]:
# define objective function to use in hyperopt method fmin

def objective(params):
    with mlflow.start_run(): # tracks for each iteration
        mlflow.set_tag("model", "xgboost") # set tag
        mlflow.set_tag("developers", "cristian & bhagabat") # set tag
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=50,
            evals=[(valid, 'validation')],
            early_stopping_rounds=5
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_valid, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [49]:
# hyper parameter tunning and logging information from each iteration to mlflow

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

[18:18:00] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.  
[0]	validation-rmse:17.59339                                                                                            
[1]	validation-rmse:14.80284                                                                                            
[2]	validation-rmse:12.66035                                                                                            
[3]	validation-rmse:11.04016                                                                                            
[4]	validation-rmse:9.82996                                                                                             
[5]	validation-rmse:8.93657                                                                                             
[6]	validation-rmse:8.28404                                                                                             
[7]	validation-rmse:7.81092     

<img src="img/03xgbexppart1.png" />

<img src="img/03xgbexppart2.png" />

<img src="img/03xgbexppart3.png" />

In [50]:
best_result

{'learning_rate': 0.20237342529787536,
 'max_depth': 81.0,
 'min_child_weight': 2.013139636617962,
 'reg_alpha': 0.009609088021234408,
 'reg_lambda': 0.011968094101381624}

In [51]:
best_result['max_depth'] = int(best_result['max_depth'])

### Train XGBOOST model and track using mlflow autolog ###

In [52]:
mlflow.xgboost.autolog() # set auto logging

params = best_result # as obtained from previous run for hyperparameter tuning
with mlflow.start_run(): # Start tracking

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
    mlflow.set_tag("model", "xgboost") # set tag
    mlflow.set_tag("developers", "cristian & bhagabat") # set tag
    mlflow.set_tag("logtype", "auto") # set tag
    mlflow.log_params(params)
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=50,
        evals=[(valid, 'validation')],
        early_stopping_rounds=5
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_valid, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

[0]	validation-rmse:17.59339
[1]	validation-rmse:14.80284
[2]	validation-rmse:12.66035
[3]	validation-rmse:11.04016
[4]	validation-rmse:9.82996
[5]	validation-rmse:8.93657
[6]	validation-rmse:8.28404
[7]	validation-rmse:7.81092
[8]	validation-rmse:7.47282
[9]	validation-rmse:7.22610
[10]	validation-rmse:7.05025
[11]	validation-rmse:6.92036
[12]	validation-rmse:6.82504
[13]	validation-rmse:6.75178
[14]	validation-rmse:6.69740
[15]	validation-rmse:6.65328
[16]	validation-rmse:6.62185
[17]	validation-rmse:6.59858
[18]	validation-rmse:6.57790
[19]	validation-rmse:6.56032
[20]	validation-rmse:6.54746
[21]	validation-rmse:6.53638
[22]	validation-rmse:6.52811
[23]	validation-rmse:6.52084
[24]	validation-rmse:6.51585
[25]	validation-rmse:6.51085
[26]	validation-rmse:6.50763
[27]	validation-rmse:6.50402
[28]	validation-rmse:6.50147
[29]	validation-rmse:6.49846
[30]	validation-rmse:6.49628
[31]	validation-rmse:6.49439
[32]	validation-rmse:6.49234
[33]	validation-rmse:6.49044
[34]	validation-rmse

2023/06/04 18:20:03 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2023/06/04 18:20:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


<img src="img/04xgbautopart1.png" />

<img src="img/04xgbautopart2.png" />

<img src="img/04xgbautopart3.png" />

### Train multiple Scikit-Learn models and track each with mlflow auto log ###

In [23]:
from sklearn.ensemble import RandomForestRegressor # , GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

In [ ]:
mlflow.sklearn.autolog() # enable auto logging

with mlflow.start_run(): # tracks for each model

    mlflow.log_param("train-data-path", "./data_2022/green_tripdata_2022-01.csv")
    mlflow.log_param("valid-data-path", "./data_2022/green_tripdata_2022-02.csv")

    mlmodel = RandomForestRegressor()
    mlmodel.fit(X_train, y_train)

    y_pred = mlmodel.predict(X_valid)
    rmse = mean_squared_error(y_valid, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

In [26]:
mlflow.sklearn.autolog() # enable auto logging

for model_class in (RandomForestRegressor) : # , GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):
    print(f'Running model: {model_class.__name__}')
    with mlflow.start_run(): # tracks for each model

        mlflow.log_param("train-data-path", "./data/green_tripdata_2022-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2022-02.csv")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_valid)
        rmse = mean_squared_error(y_valid, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

TypeError: 'ABCMeta' object is not iterable

In [55]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

In [54]:
mlflow.sklearn.autolog() # enable auto logging

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):
    print(f'Running model: {model_class.__name__}')
    with mlflow.start_run(): # tracks for each model

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_valid)
        rmse = mean_squared_error(y_valid, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

Running model: RandomForestRegressor


KeyboardInterrupt: 

In [56]:
mlflow.sklearn.autolog() # enable auto logging

with mlflow.start_run(): # tracks for each model

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    mlmodel = LinearSVR()
    mlmodel.fit(X_train, y_train)

    y_pred = mlmodel.predict(X_valid)
    rmse = mean_squared_error(y_valid, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [57]:
import subprocess

# Launch MLflow UI
subprocess.Popen(["mlflow", "ui"])

<Popen: returncode: None args: ['mlflow', 'ui']>

[2023-06-04 18:37:23 +0000] [8362] [INFO] Starting gunicorn 20.1.0
[2023-06-04 18:37:23 +0000] [8362] [INFO] Listening at: http://127.0.0.1:5000 (8362)
[2023-06-04 18:37:23 +0000] [8362] [INFO] Using worker: sync
[2023-06-04 18:37:23 +0000] [8363] [INFO] Booting worker with pid: 8363
[2023-06-04 18:37:23 +0000] [8364] [INFO] Booting worker with pid: 8364
[2023-06-04 18:37:23 +0000] [8365] [INFO] Booting worker with pid: 8365
[2023-06-04 18:37:23 +0000] [8366] [INFO] Booting worker with pid: 8366
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 290, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/home/ubuntu/anaconda3/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 383, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/ubuntu/anaconda3/lib/python3.10/site-packages/mlflow/store/track